In [1]:
import pandas as pd
import re 
import spacy 
import os 
import numpy as np
import altair as alt
import textdistance
from sklearn.preprocessing import MultiLabelBinarizer
alt.data_transformers.enable('json')
pd.options.display.max_colwidth = 100

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
extract = pd.read_excel("../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/01_resume_scan_data/manual_extraction_template.xlsx")
extract.columns = extract.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [4]:
extract.head(1)

,employee_code,employee_name,found,work1_title,work1_company,work1_time,work2_title,work2_company,work2_time,work3_title,...,education2_concentration,education2_country,education3_school,education3_degree,education3_concentration,education3_country,education4_school,education4_degree,education4_concentration,education4_country
0,BDP,"Abdulali, Kaizar",clara,Sales Manager (Samsung Experience Store),Glentel Inc.,Nov 2015 - Present,Sales Manager (Wireless Wave & T-Booth),Glentel Inc.,Aug 2010 - Nov 2015,Sales Associate (Wireless Wave & T-Booth),...,Economic,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# extract = extract.fillna('blank')

In [6]:
# extract.work1_title = extract.work1_title.str.lower()

In [7]:
extract["work1_title"] = extract["work1_title"].astype("string")

In [8]:
def preprocess(text): 
    
    text = str(text)
    text = text.lower()
    text = text.strip()
    text = re.sub('\W+',' ', text)
    text = re.sub("[\d-]", '', text)
    
    # Replace a sequence of whitespaces by a single whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove other strange characters
    text = re.sub(r'''[\*\~]+''', "", text)
    
    #hard code changes 
    if text == 'nan': text = "not_specified"
    if text == 'art': text = 'arts'
    if text == 'liberal arts': text = 'arts'
    if text == 'arts marketing': text = 'marketing'
    if text == 'arts psychology': text = 'psychology'
    if text == 'general arts': text = 'arts'
    
    #if they list two take the first degree
    text = re.split(" and ", text)[0]
    
    #replace word 
    text = re.sub("not mention.+", "not_specified", text)
    text = re.sub("advanced ", '', text)
    text = re.sub("degree", '', text)
    text = re.sub("certificate", '', text)
    text = re.sub("administrat", '', text)
    text = re.sub("management", '', text)
    text = re.sub("progress", '', text)
    text = re.sub("mba", 'business', text)
    text = re.sub("bachelor", '', text)
    text = re.sub("commerce", 'business', text)
    text = re.sub("hr", 'human resource', text)
    text = re.sub("mortgage", 'finance', text)
    text = re.sub("securities", 'finance', text)
    text = re.sub("radio", 'audio', text)
    text = re.sub("tv ", 'television', text)
    text = re.sub('television', 'film', text)
    text = re.sub('health care', 'healthcare', text)
    text = re.sub('hospital ', 'healthcare', text)
    text = re.sub('computing', 'computer', text)
    text = re.sub('techonology', 'technology', text)
    text = re.sub('socialogy', 'sociology', text)
    text = re.sub('child', 'education', text)
    text = re.sub('office', 'general', text)
    text = re.sub('general arts', 'arts', text)
    text = re.sub('public', 'general', text)
    text = re.sub('o s s d', 'general', text)
    text = re.sub('informatics security', 'computer', text) 
    text = re.sub("bank", 'business', text) 
    
    #group degree based off of key words
    if re.search("law", text): text = "law"
    if re.search("dental", text): text = "dental"
    if re.search("financ", text): text = "finance"
    if re.search("account", text): text = "accounting"
    if re.search("human resource", text): text = "human resource"
    if re.search("marketing", text): text = "marketing"
    if re.search("business", text): text = "business"
    if re.search("audio", text): text = "audio technician"
    if re.search("film", text): text = "film production"
    if re.search('healthcare', text): text = "healthcare"
    if re.search('counsel', text): text = "healthcare"
    if re.search('engineering', text): text = "engineering"
    if re.search('communication', text): text = "communication"
    if re.search('education', text): text = 'education'
    if re.search('kinesiology', text): text = 'kinesiology'
    if re.search('computer', text): text = 'computer systems'
    if re.search('software', text): text = 'computer systems'
    if re.search('police', text): text = 'law'
    if re.search('science', text): text = 'science'
    if re.search('english', text): text = 'english'
    if re.search('paralegal', text): text = 'law'
    if re.search('social service', text): text = 'arts'
    if re.search('esthetician', text): text = 'health'
    if re.search('fitness', text): text = 'health'
    if re.search('carpent', text): text = 'blue collar'
    if re.search('plumb', text): text = 'blue collar'
    if re.search('electric', text): text = 'blue collar'
    if re.search('crane', text): text = 'blue collar'
    if re.search('media', text): text = 'interactive arts and technology'
    if re.search('technology', text): text = 'interactive arts and technology'
    if re.search('game', text): text = 'interactive arts and technology'
    if re.search('entertainment', text): text = 'interactive arts and technology'
    if re.search('information', text): text = 'interactive arts and technology'
    if re.search('digital', text): text = 'interactive arts and technology'
    if re.search('medica', text): text = "healthcare"
    if re.search('nurs', text): text = 'healthcare'
    if re.search('pharma', text): text = 'healthcare'
    if re.search('linguist', text): text = 'english'
    if re.search('translation', text): text = 'arts'
    if re.search('general', text): text = 'general'
    if re.search('photo', text): text = 'interactive arts and technology'
    if re.search('screen', text): text = 'interactive arts and technology'
    if re.search('profession', text): text = 'arts'
    if re.search('liberal', text): text = 'arts'
    if re.search('graphic', text): text = 'interactive arts and technology'
    
    
    if text == 'not mention': text = "not_specified"
    
    
    #text = re.sub("arts ", '', text)
    
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

In [9]:
# extract["education1_concentration_clean"] = extract["education1_concentration_clean"].fillna('not mentioned')

In [10]:
for i in range(1,4):
    col_num = i
    extract["education"+str(col_num)+"_concentration_clean"] = extract["education"+str(col_num)+"_concentration"].apply(preprocess)
    

In [11]:
# extract["education"+str(col_num)+"_concentration_clean"].to_clipboard()

In [12]:
col_num = 1
alt.Chart(extract).mark_bar().encode(
    x="count()",
    y='education'+str(col_num)+'_concentration_clean'

)

alt.Chart(...)

In [13]:
#could have made this more efficent
def concentration_labeling(con):
    con = str(con)
    if any(x in con for x in ['business']):
        label = "business_concentration"
    elif con == 'arts': label = "arts_concentration"
    elif con == 'interactive arts and technology': label = 'interactive_arts_and_technology_concentration'
    elif any(x in con for x in ['computer systems']):
        label = 'computer_systems_concentration'
    elif any(x in con for x in ['sociology']):
        label = 'arts_concentration'
    elif any(x in con for x in ['finance']):
        label = 'finance_concentration'
    elif any(x in con for x in ['marketing']):
        label = 'marketing_concentration'
    elif any(x in con for x in ['engineering']):
        label = 'engineering_concentration'
    elif any(x in con for x in ['psychology']):
        label = 'arts_concentration'
    elif any(x in con for x in ['general']):
        label = 'general_concentration'
    elif any(x in con for x in ['human resource']):
        label = 'human_resource_concentration'
    elif any(x in con for x in ['accounting']):
        label = 'accounting_concentration'
    elif any(x in con for x in ['audio technician']):
        label = 'interactive_arts_and_technology_concentration'
    elif any(x in con for x in ['education']):
        label = 'arts_concentration'
    elif any(x in con for x in ['communication']):
        label = 'arts_concentration'
    elif any(x in con for x in ['english']):
        label = 'arts_concentration'
    elif any(x in con for x in ['criminology']):
        label = 'arts_concentration'
    elif any(x in con for x in ['economics']):
        label = 'business_concentration'
    elif any(x in con for x in ['not_specified']):
        label = 'not_specified_concentration'
    else:
        label = "other_concentration" 
    return label

In [14]:
for i in range(1,4):
    col_num = i
    extract["education"+str(col_num)+"_label"] = extract["education"+str(col_num)+"_concentration_clean"].apply(concentration_labeling)
    

In [15]:
col_num=1
alt.Chart(extract).mark_bar().encode(
    x="count()",
    y='education'+str(col_num)+'_label'
)

alt.Chart(...)

In [16]:
extract.head()

,employee_code,employee_name,found,work1_title,work1_company,work1_time,work2_title,work2_company,work2_time,work3_title,...,education4_school,education4_degree,education4_concentration,education4_country,education1_concentration_clean,education2_concentration_clean,education3_concentration_clean,education1_label,education2_label,education3_label
0,BDP,"Abdulali, Kaizar",clara,Sales Manager (Samsung Experience Store),Glentel Inc.,Nov 2015 - Present,Sales Manager (Wireless Wave & T-Booth),Glentel Inc.,Aug 2010 - Nov 2015,Sales Associate (Wireless Wave & T-Booth),...,NaN,NaN,NaN,NaN,business,economic,not_specified,business_concentration,other_concentration,not_specified_concentration
1,F6Q,"Avila-Gil, Mario",clara,Sales Manager,Glentel/ Wireless ETC,NaN,Customer Service Consultant,Cityfone/ Rogers Communications,Nov 2009 - Sep 2011,,...,NaN,NaN,NaN,NaN,not_specified,not_specified,not_specified,not_specified_concentration,not_specified_concentration,not_specified_concentration
2,MRN,"Amini, Kamyar",clara,Sales Associate/ Educator,THE BRICK,2008-2010,Reception/Cashier,CHAIRMANS MANUFACTURING,2006 - 2008,NaN,...,NaN,NaN,NaN,NaN,business,not_specified,not_specified,business_concentration,not_specified_concentration,not_specified_concentration
3,MXO,"Awan, Shoaib",clara,Team Leader,TRAVELEX GROUP CANADA,Nov 2014 - Present,Support Trainer,TRAVELEX GROUP CANADA,Jan 2013 - Oct 2014,ATM Coordinator / Operations,...,NaN,NaN,NaN,NaN,not_specified,not_specified,not_specified,not_specified_concentration,not_specified_concentration,not_specified_concentration
4,N0Y,"Al-Khulaidy, Maha",clara,Sales Associate,Sportcheck FGL Sports,February 2019 - Present,Multi-Channel Sales Associate,Best Buy,Nov 2017 - Jan 2019,Customer Service Consultant,...,NaN,NaN,NaN,NaN,science,not_specified,not_specified,other_concentration,not_specified_concentration,not_specified_concentration


In [17]:
result = []
for i in range(0, len(extract['education1_label'])):
    lst = []
    if (str(extract['education1_label'][i]) == 'not_specified_concentration' and str(extract['education2_label'][i]) == 'not_specified_concentration' and str(extract['education3_label'][i]) == 'not_specified_concentration'):
        lst.append("no_education")
    if extract['education1_label'][i] != 'not_specified_concentration':
        lst.append(extract['education1_label'][i])
    if extract['education2_label'][i] != 'not_specified_concentration':
        lst.append(extract['education2_label'][i])
    if extract['education3_label'][i] != 'not_specified_concentration':
        lst.append(extract['education3_label'][i])
    result.append(lst)


In [18]:
extract["education_list"] = result

In [19]:
extract

,employee_code,employee_name,found,work1_title,work1_company,work1_time,work2_title,work2_company,work2_time,work3_title,...,education4_degree,education4_concentration,education4_country,education1_concentration_clean,education2_concentration_clean,education3_concentration_clean,education1_label,education2_label,education3_label,education_list
0,BDP,"Abdulali, Kaizar",clara,Sales Manager (Samsung Experience Store),Glentel Inc.,Nov 2015 - Present,Sales Manager (Wireless Wave & T-Booth),Glentel Inc.,Aug 2010 - Nov 2015,Sales Associate (Wireless Wave & T-Booth),...,NaN,NaN,NaN,business,economic,not_specified,business_concentration,other_concentration,not_specified_concentration,"[business_concentration, other_concentration]"
1,F6Q,"Avila-Gil, Mario",clara,Sales Manager,Glentel/ Wireless ETC,NaN,Customer Service Consultant,Cityfone/ Rogers Communications,Nov 2009 - Sep 2011,,...,NaN,NaN,NaN,not_specified,not_specified,not_specified,not_specified_concentration,not_specified_concentration,not_specified_concentration,[no_education]
2,MRN,"Amini, Kamyar",clara,Sales Associate/ Educator,THE BRICK,2008-2010,Reception/Cashier,CHAIRMANS MANUFACTURING,2006 - 2008,NaN,...,NaN,NaN,NaN,business,not_specified,not_specified,business_concentration,not_specified_concentration,not_specified_concentration,[business_concentration]
3,MXO,"Awan, Shoaib",clara,Team Leader,TRAVELEX GROUP CANADA,Nov 2014 - Present,Support Trainer,TRAVELEX GROUP CANADA,Jan 2013 - Oct 2014,ATM Coordinator / Operations,...,NaN,NaN,NaN,not_specified,not_specified,not_specified,not_specified_concentration,not_specified_concentration,not_specified_concentration,[no_education]
4,N0Y,"Al-Khulaidy, Maha",clara,Sales Associate,Sportcheck FGL Sports,February 2019 - Present,Multi-Channel Sales Associate,Best Buy,Nov 2017 - Jan 2019,Customer Service Consultant,...,NaN,NaN,NaN,science,not_specified,not_specified,other_concentration,not_specified_concentration,not_specified_concentration,[other_concentration]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,MUD,"Ryoo, Sungtak",thomas,Technician,uBreakiFix,July 2018 - Present,Production Worker,Toyota,April 2018 - July 2018,Manager,...,NaN,NaN,NaN,law,not_specified,not_specified,other_concentration,not_specified_concentration,not_specified_concentration,[other_concentration]
285,L46,"Kapasi, Nikunj",thomas,ECOMMERCE ANALYST,DOT LIGHTING,Jan 2019 - Present,SALES ASSOCIATE,TBOOTH WIRELESS,Feb 2017 - Jan 2019,E-COMMERCE COORDINATOR,...,NaN,NaN,NaN,business,science,not_specified,business_concentration,other_concentration,not_specified_concentration,"[business_concentration, other_concentration]"
286,N87,"Mcinnis, Peter",thomas,Order Selector,LOBLAWS,November 2017 - PRESENT,Manager,LINEN CHEST,February 2015 - March 2018,Customer Service Representative,...,NaN,NaN,NaN,not_specified,not_specified,not_specified,not_specified_concentration,not_specified_concentration,not_specified_concentration,[no_education]
287,ND9,"Gamble, Justin",thomas,Cashier,Save On Foods,March 2017 - Present,Photographer/Illustrator,Gamble Visuals,June 2016 - Present,Sales Associate,...,NaN,NaN,NaN,not_specified,not_specified,not_specified,not_specified_concentration,not_specified_concentration,not_specified_concentration,[no_education]


In [20]:
mlb = MultiLabelBinarizer()
extract = extract.join(pd.DataFrame(mlb.fit_transform(extract.pop('education_list')),
                                   columns = mlb.classes_,
                                   index= extract.index))

In [21]:
extract.columns

Index(['employee_code', 'employee_name', 'found', 'work1_title',
       'work1_company', 'work1_time', 'work2_title', 'work2_company',
       'work2_time', 'work3_title', 'work3_company', 'work3_time',
       'work4_title', 'work4_company', 'work4_time', 'work5_title',
       'work5_company', 'work5_time', 'work6_title', 'work6_company',
       'work6_time', 'work7_title', 'work7_company', 'work7_time',
       'education1_school', 'education1_degree', 'education1_concentration',
       'education1_country', 'education2_school', 'education2_degree',
       'education2_concentration', 'education2_country', 'education3_school',
       'education3_degree', 'education3_concentration', 'education3_country',
       'education4_school', 'education4_degree', 'education4_concentration',
       'education4_country', 'education1_concentration_clean',
       'education2_concentration_clean', 'education3_concentration_clean',
       'education1_label', 'education2_label', 'education3_label',
       

In [22]:
extract = extract.drop(['found', 'work1_title',
       'work1_company', 'work1_time', 'work2_title', 'work2_company',
       'work2_time', 'work3_title', 'work3_company', 'work3_time',
       'work4_title', 'work4_company', 'work4_time', 'work5_title',
       'work5_company', 'work5_time', 'work6_title', 'work6_company',
       'work6_time', 'work7_title', 'work7_company', 'work7_time',
       'education1_school', 'education1_degree', 'education1_concentration',
       'education1_country', 'education2_school', 'education2_degree',
       'education2_concentration', 'education2_country', 'education3_school',
       'education3_degree', 'education3_concentration', 'education3_country',
       'education4_school', 'education4_degree', 'education4_concentration',
       'education4_country', 'education1_concentration_clean',
       'education2_concentration_clean', 'education3_concentration_clean'], axis=1)

In [23]:
extract.query('employee_code == "MRU"')

,employee_code,employee_name,education1_label,education2_label,education3_label,accounting_concentration,arts_concentration,business_concentration,computer_systems_concentration,engineering_concentration,finance_concentration,general_concentration,human_resource_concentration,interactive_arts_and_technology_concentration,marketing_concentration,no_education,other_concentration
247,MRU,"Williams, Dave",interactive_arts_and_technology_concentration,not_specified_concentration,not_specified_concentration,0,0,0,0,0,0,0,0,1,0,0,0


In [24]:
extract.to_csv('../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/06_clean_data/education_concentration_condensed.csv')

In [25]:
extract.describe()

,accounting_concentration,arts_concentration,business_concentration,computer_systems_concentration,engineering_concentration,finance_concentration,general_concentration,human_resource_concentration,interactive_arts_and_technology_concentration,marketing_concentration,no_education,other_concentration
count,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000
mean,0.034602,0.145329,0.245675,0.079585,0.051903,0.051903,0.200692,0.024221,0.062284,0.038062,0.179931,0.238754
std,0.183087,0.353043,0.431233,0.271119,0.222216,0.222216,0.401213,0.154003,0.242089,0.191679,0.384796,0.427062
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
